In [8]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('/content/subset.csv')

# Convert 'true' to 1 and 'false' to 0 in the 'label' column
df['label'] = df['label'].replace({'true': 1, 'false': 0})

# Save the modified DataFrame to a new CSV file
df.to_csv('path_to_your_modified_file.csv', index=False)



In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import XLNetTokenizer

# Load the combined dataset
df = pd.read_csv('/content/subset.csv')

# Split the data into train and test sets
train_df, test_df = train_test_split(df, test_size=0.5, random_state=42, stratify=df['label'])

# Load the tokenizer
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')

# Ensure all text entries are strings
train_texts = train_df['text'].astype(str).tolist()
test_texts = test_df['text'].astype(str).tolist()
train_labels = train_df['label'].tolist()
test_labels = test_df['label'].tolist()

# Tokenize the text
def tokenize_texts(texts):
    return tokenizer(
        texts,
        padding=True,
        truncation=True,
        max_length=512,
        return_tensors='pt'
    )

train_encodings = tokenize_texts(train_texts)
test_encodings = tokenize_texts(test_texts)

In [10]:
import torch
from transformers import XLNetForSequenceClassification

# Load the model
model = XLNetForSequenceClassification.from_pretrained('xlnet-base-cased', num_labels=2)

# Move the model to GPU if available
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'logits_proj.weight', 'sequence_summary.summary.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


XLNetForSequenceClassification(
  (transformer): XLNetModel(
    (word_embedding): Embedding(32000, 768)
    (layer): ModuleList(
      (0-11): 12 x XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (layer_1): Linear(in_features=768, out_features=3072, bias=True)
          (layer_2): Linear(in_features=3072, out_features=768, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (activation_function): GELUActivation()
        )
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (sequence_summary): SequenceSummary(
    (summary): Linear(in_features=768, out_features=768, bias=True)
    (activation): Tanh()
    (first_dropout): Identity()
    (last

In [11]:
import torch
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from transformers import XLNetForSequenceClassification, get_linear_schedule_with_warmup
from sklearn.metrics import accuracy_score, f1_score

# Assuming 'train_labels' and 'test_labels' are lists of 'true'/'false' strings
# Here, you need to define or import 'train_labels' and 'test_labels'
train_labels = [1 if label == 'true' else 0 for label in train_labels]
test_labels = [1 if label == 'true' else 0 for label in test_labels]

# Convert labels to tensors
train_labels = torch.tensor(train_labels)
test_labels = torch.tensor(test_labels)

# Assuming 'train_encodings' and 'test_encodings' are dicts with 'input_ids' and 'attention_mask'
# Here, you need to define or import 'train_encodings' and 'test_encodings'
train_dataset = TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'], train_labels)
test_dataset = TensorDataset(test_encodings['input_ids'], test_encodings['attention_mask'], test_labels)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

# Initialize the XLNet model for sequence classification
model = XLNetForSequenceClassification.from_pretrained('xlnet-base-cased', num_labels=2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Optimizer and Scheduler
optimizer = optim.AdamW(model.parameters(), lr=2e-5)
total_steps = len(train_loader) * 4  # Assuming 4 epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

# Training function
# Training function
def train(model, train_loader):
    model.train()
    total_loss = 0
    for batch in train_loader:
        input_ids, attention_mask, labels = [item.to(device) for item in batch]
        model.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)  # Fixed the typo here
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        scheduler.step()
    return total_loss / len(train_loader)


# Evaluation function
def evaluate(model, test_loader):
    model.eval()
    predictions, true_labels = [], []
    with torch.no_grad():
        for batch in test_loader:
            input_ids, attention_mask, labels = [item.to(device) for item in batch]
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            predictions.extend(torch.argmax(logits, dim=-1).cpu().numpy())
            true_labels.extend(labels.cpu().numpy())
    accuracy = accuracy_score(true_labels, predictions)
    f1 = f1_score(true_labels, predictions, average='weighted')
    return accuracy, f1

# Training and Evaluation
epochs = 1
for epoch in range(epochs):
    train_loss = train(model, train_loader)
    print(f"Epoch {epoch + 1}, Loss: {train_loss}")

accuracy, f1 = evaluate(model, test_loader)
print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")


Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'logits_proj.weight', 'sequence_summary.summary.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1, Loss: 0.02302498549374471
Accuracy: 1.0
F1 Score: 1.0
